In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import json

def calcular_media_equipo(equipo):
    media = {}
    for campeon in equipo:
        for key, value in campeon.items():
            if key != "ID" and key != "Key" and key != "Name":
                if key not in media:
                    media[key] = value
                else:
                    media[key] += value
    num_campeones = len(equipo)
    for key in media:
        media[key] /= num_campeones
    return list(media.values())

def cargar_datos_desde_json(ruta_archivo):
    X = []
    y = []
    with open(ruta_archivo, 'r') as file:
        data = json.load(file)
    for partida in data:
        resultado = partida["result"]
        for equipo in [partida["blue_team"], partida["red_team"]]:
            X.append(calcular_media_equipo(equipo))
            y.append(resultado)
    return np.array(X), np.array(y)

# Cargar datos desde el archivo JSON
X, y = cargar_datos_desde_json("partidasConCaracteristicas.json")

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Valores de C a probar
valores_c = [0.001, 0.01, 0.1, 1, 2, 10]

# Iterar sobre los valores de C
for c in valores_c:
    # Inicializar el clasificador SVC con el valor de C actual
    svc = SVC(kernel='linear', C=c)

    # Entrenar el modelo
    svc.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = svc.predict(X_test)

    # Calcular la precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    print("Precisión del modelo SVC con C={}: {:.4f}".format(c, precision))


Precisión del modelo SVC con C=0.001: 0.5370
Precisión del modelo SVC con C=0.01: 0.5370
Precisión del modelo SVC con C=0.1: 0.5370
Precisión del modelo SVC con C=1: 0.5370
Precisión del modelo SVC con C=2: 0.5370
Precisión del modelo SVC con C=10: 0.5370


In [4]:
import json

with open('arraysPartidasResultado.json') as file:
    data_results = json.load(file)

with open('caracteristicas_campeones.json') as file:
    data_champions = json.load(file)


In [5]:
import pandas as pd

# Transformamos la lista de campeones en un diccionario para facilitar el acceso a sus características por clave
champions_features = {champ['Key']: champ for champ in data_champions}
# Calculamos las medias de las características
features_to_average = ['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']
average_features = {feature: sum(champ[feature] for champ in champions_features.values()) / len(champions_features) for feature in features_to_average}
# Crear un DataFrame vacío para almacenar los datos combinados
combined_data = []

# Iteramos sobre cada partida en data_results
for match in data_results:
    # Extrayendo el resultado del partido y los IDs de los campeones
    match_result = match[0]
    champion_ids = match[1:]

    # Lista para almacenar las características combinadas de los campeones de la partida
    match_features = []

    # Sumar características de todos los campeones en la partida
    for champ_id in champion_ids:
        # Si el campeón está en nuestro diccionario de características, agregamos sus datos
        if champ_id in champions_features:
            champ_data = champions_features[champ_id]
            match_features.extend([
                champ_data['Difficulty'],
                champ_data['HP'],
                champ_data['Armor'],
                champ_data['HPRegen'],
                champ_data['AttackDamage'],
                champ_data['AttackSpeed']
            ])
        else:

            # Usamos las medias si el campeón no está en el diccionario
            match_features.extend([average_features[feat] for feat in features_to_average])

    # Agregamos el resultado del partido y las características acumuladas a los datos combinados
    combined_data.append([match_result] + match_features)

# Convertimos la lista combinada en un DataFrame para su manipulación y visualización
column_names = ['Result'] + [f'Champ{i}_{feat}' for i in range(1, 11)
                             for feat in ['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']]
df_combined = pd.DataFrame(combined_data, columns=column_names)

# Mostramos las primeras filas del DataFrame para verificar
df_combined.head()

,Result,Champ1_Difficulty,Champ1_HP,Champ1_Armor,Champ1_HPRegen,Champ1_AttackDamage,Champ1_AttackSpeed,Champ2_Difficulty,Champ2_HP,Champ2_Armor,...,Champ9_Armor,Champ9_HPRegen,Champ9_AttackDamage,Champ9_AttackSpeed,Champ10_Difficulty,Champ10_HP,Champ10_Armor,Champ10_HPRegen,Champ10_AttackDamage,Champ10_AttackSpeed
0,1,8.0,630.0,33.0,8.5,64.0,0.625,7.0,655.0,27.0,...,29.835366,6.41311,58.920732,0.649543,3.0,610.0,47.0,8.5,55.0,0.644
1,0,8.0,596.0,22.0,5.5,55.0,0.625,7.0,600.0,20.0,...,28.000000,3.50000,50.000000,0.625000,5.0,575.0,27.0,9.0,61.0,0.645
2,1,4.0,650.0,40.0,6.0,64.0,0.658,5.0,670.0,38.0,...,29.000000,3.75000,62.000000,0.679000,5.0,610.0,30.0,5.0,62.0,0.635
3,0,3.0,660.0,35.0,8.0,69.0,0.665,7.0,658.0,30.0,...,29.835366,6.41311,58.920732,0.649543,5.0,610.0,30.0,5.0,62.0,0.635
4,0,3.0,660.0,35.0,8.0,69.0,0.665,8.0,558.0,19.0,...,24.000000,4.00000,62.000000,0.625000,4.0,646.0,35.0,8.5,68.0,0.644


In [6]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separar los datos en variables independientes (X) y la variable dependiente (y)
X = df_combined.iloc[:, 1:].values  # Todas las filas, todas las columnas excepto la primera
y = df_combined.iloc[:, 0].values  # Todas las filas, solo la primera columna (resultado del partido)

# Normalizar las características
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# Guardar el scaler para su uso posterior
joblib.dump(scaler, 'scaler.gz')
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mostramos las dimensiones de los conjuntos de entrenamiento y prueba para confirmar
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)


(((11026, 60), (11026,)), ((2757, 60), (2757,)))

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Asumiendo df_combined ya está definido y cargado correctamente
X = df_combined.drop('Result', axis=1)  # características
y = df_combined['Result']  # etiqueta

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo LinearSVC
model = LinearSVC(max_iter=10000)  # Aumentar max_iter si es necesario por convergencia
model.fit(X_train_scaled, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Calcular y mostrar la precisión
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the LinearSVC model:", accuracy)


Accuracy of the LinearSVC model: 0.5523579201934704


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Asumiendo df_combined ya está definido y cargado correctamente
X = df_combined.drop('Result', axis=1)  # características
y = df_combined['Result']  # etiqueta

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Diferentes valores de C para explorar
C_values = [0.001, 0.01, 0.1, 1, 10]

# Almacenar resultados para cada valor de C
results = []

for C in C_values:
    model = LinearSVC(C=C, max_iter=10000)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    results.append((C, accuracy))
    print(f"Accuracy with C={C}: {accuracy}")

# Mostrar todos los resultados
for C, accuracy in results:
    print(f"C = {C}: Accuracy = {accuracy}")


Accuracy with C=0.001: 0.550665054413543
Accuracy with C=0.01: 0.5521160822249093
Accuracy with C=0.1: 0.5523579201934704
Accuracy with C=1: 0.5523579201934704
Accuracy with C=10: 0.5530834340991536
C = 0.001: Accuracy = 0.550665054413543
C = 0.01: Accuracy = 0.5521160822249093
C = 0.1: Accuracy = 0.5523579201934704
C = 1: Accuracy = 0.5523579201934704
C = 10: Accuracy = 0.5530834340991536


C:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
